In [14]:
import requests # type: ignore
import pandas as pd # type: ignore
import numpy as np # type: ignore
import json

In [15]:
# cmsc_data = requests.get(f"https://planetterp.com/api/v1/courses?department=CMSC").json()
# with open('cmsc_data.json', 'w') as f:
#     json.dump(cmsc_data, f)

In [16]:
cmsc_data = requests.get(f"https://planetterp.com/api/v1/courses?department=CMSC").json()

cmsc_df = pd.DataFrame(cmsc_data)
cmsc_df = cmsc_df.sort_values(by=['course_number'])

cmsc_df = cmsc_df[cmsc_df['course_number'].str.startswith(('1','2','3','4'))]
cmsc_df.drop(columns=['professors', 'description', 'is_recent', 'geneds', 'title'], inplace=True)
cmsc_df.dropna(subset=['average_gpa'], inplace=True)

average_cmsc_gpa = cmsc_df['average_gpa'].sum() / len(cmsc_df)
courses = cmsc_df['name'].values

print(average_cmsc_gpa)
print(cmsc_df)

2.9276091270621403
    average_gpa department course_number      name  credits
2      3.496197       CMSC           100   CMSC100      1.0
39     2.397251       CMSC           106   CMSC106      4.0
16     2.527072       CMSC           122   CMSC122      3.0
40     2.487990       CMSC           131   CMSC131      4.0
41     2.602482       CMSC           132   CMSC132      4.0
88     3.365746       CMSC          132H  CMSC132H      4.0
8      3.152688       CMSC          198C  CMSC198C      NaN
9      2.651282       CMSC          198D  CMSC198D      1.0
10     0.800000       CMSC          198E  CMSC198E      1.0
4      2.471644       CMSC           216   CMSC216      4.0
11     2.668913       CMSC           250   CMSC250      4.0
48     3.448966       CMSC          250H  CMSC250H      4.0
23     2.946301       CMSC          289I  CMSC289I      3.0
76     2.700000       CMSC          289J  CMSC289J      NaN
71     3.388555       CMSC           320   CMSC320      3.0
5      2.790153      

In [17]:
#For a specific course 
df = pd.DataFrame()
#Takes around 50s for me to run LMFAO
for x in courses:
    grade_data = requests.get(f"https://api.planetterp.com/v1/grades?course={x}").json()

    df1 = pd.DataFrame(grade_data)
    df1 = df1.groupby(['professor', 'semester']).sum().reset_index()
    df1.drop(columns=['section', "Other", "W"], inplace=True)
    df1 = df1.sort_values(by=['semester'])
    #print(df1.columns)
    df = pd.concat([df1, df])
df = df.iloc[::-1]
df.drop(columns=['professor', 'course'], inplace=True)


In [ ]:
df = df.sort_values(by=['semester'])
df = df.groupby(['semester']).sum().reset_index()
print(df)

   semester    A+     A    A-   B+     B   B-   C+    C   C-  D+    D  D-    F
0    201201   117   371   185  122   415  186  101  320  150   6  113  18  128
1    201208   117   398   163  182   435  214  154  321  155  31  121  17  132
2    201301   141   362   233  174   490  182  126  311  130  33  120  27  125
3    201308   165   451   226  206   479  255  151  276  157  14  109  11  137
4    201401   155   427   224  184   475  202  145  311  167  20  121  24  106
5    201408   138   397   221  184   479  181  123  255  124   1  126  18  101
6    201501   102   289   144  158   388  170  129  222  140  14  109  20  143
7    201508   316   678   334  387   721  306  284  427  261   5  209  21  244
8    201601   199   675   364  385   723  393  273  427  308  20  190  27  202
9    201608   327   782   408  495   763  405  335  501  297  20  233  51  232
10   201701   308   811   420  465   828  452  378  483  341  33  209  46  173
11   201708   327   965   548  540   877  477  374  

In [ ]:
#Code if we want to do yearly
#df['year'] = df['semester'].astype(str).str[:4]
#df = df.groupby('year').sum().reset_index()
#df.drop(columns=['semester'], inplace=True)
#print(df)

In [ ]:
#Calculate GPA per semester or year
scale = np.array([4.0, 4.0, 3.7, 3.3, 3.0, 2.7, 2.3, 2.0, 1.7, 1.3, 1.0, 0.7, 0.0])
grades = df.iloc[:, 1:]
print(grades)
total_pts = np.dot(grades, scale)


      A+     A    A-   B+     B   B-   C+    C   C-  D+    D  D-    F
0    117   371   185  122   415  186  101  320  150   6  113  18  128
1    117   398   163  182   435  214  154  321  155  31  121  17  132
2    141   362   233  174   490  182  126  311  130  33  120  27  125
3    165   451   226  206   479  255  151  276  157  14  109  11  137
4    155   427   224  184   475  202  145  311  167  20  121  24  106
5    138   397   221  184   479  181  123  255  124   1  126  18  101
6    102   289   144  158   388  170  129  222  140  14  109  20  143
7    316   678   334  387   721  306  284  427  261   5  209  21  244
8    199   675   364  385   723  393  273  427  308  20  190  27  202
9    327   782   408  495   763  405  335  501  297  20  233  51  232
10   308   811   420  465   828  452  378  483  341  33  209  46  173
11   327   965   548  540   877  477  374  527  331  20  284  13  249
12   339   909   545  568   900  547  458  532  306  12  245  26  227
13   367   992   712

In [ ]:
df['total'] = df[['A+', 'A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-', 'D+', 'D', 'D-', 'F']].sum(axis=1)
df['average_gpa'] = total_pts / df['total'].values
print(df)

#after this point we prob do graphs, can reuse code for other departments now

   semester    A+     A    A-   B+     B   B-   C+    C   C-  D+    D  D-  \
0    201201   117   371   185  122   415  186  101  320  150   6  113  18   
1    201208   117   398   163  182   435  214  154  321  155  31  121  17   
2    201301   141   362   233  174   490  182  126  311  130  33  120  27   
3    201308   165   451   226  206   479  255  151  276  157  14  109  11   
4    201401   155   427   224  184   475  202  145  311  167  20  121  24   
5    201408   138   397   221  184   479  181  123  255  124   1  126  18   
6    201501   102   289   144  158   388  170  129  222  140  14  109  20   
7    201508   316   678   334  387   721  306  284  427  261   5  209  21   
8    201601   199   675   364  385   723  393  273  427  308  20  190  27   
9    201608   327   782   408  495   763  405  335  501  297  20  233  51   
10   201701   308   811   420  465   828  452  378  483  341  33  209  46   
11   201708   327   965   548  540   877  477  374  527  331  20  284  13   